In [1]:
from PIL import Image
import numpy as np

# Cargar la imagen
imagen = Image.open("a.png")
imagen = imagen.convert("RGB")  # Convierte a RGB para eliminar canal alfa
# Redimensionar para que coincida con el tamaño esperado por el modelo
# Por ejemplo, si tu modelo espera 32x32, ajusta la imagen
#imagen = imagen.resize((400, 400))  # Redimensionar a 32x32

# Convertir a array numpy y escalar valores a [0, 1] si es necesario
imagen_array = np.array(imagen) / 255.0  # Normalizar valores entre 0 y 1

# Asegúrate de que tenga el número correcto de dimensiones (batch, height, width, channels)
imagen_array = np.expand_dims(imagen_array, axis=0)  # Agregar dimensión de lote

In [16]:
print(imagen)

<PIL.Image.Image image mode=RGB size=1920x1080 at 0x1B0F459AE20>


In [17]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
# Crear un modelo convolucional con cinco filtros y diferentes tamaños de kernel
model = keras.Sequential([
    # Primera capa de convolución con 5 filtros y kernel 3x3
    layers.Conv2D(1, (3, 3), activation='relu', input_shape=(1080, 1920, 3)),  # 5 filtros de tamaño 3x3
    layers.MaxPooling2D((2, 2)),  # Pooling para reducir tamaño

    # Segunda capa de convolución con 5 filtros y kernel 5x5
    layers.Conv2D(1, (5, 5), activation='relu'),  # 5 filtros de tamaño 5x5
    layers.MaxPooling2D((2, 2)),  # Pooling para reducir tamaño

    # Aquí terminamos con las capas convolucionales para evitar reducción excesiva

    # Aplanar y capas densas
    layers.Flatten(),  # Aplana para pasar a capas densas
    layers.Dense(64, activation='relu'),  # Capa completamente conectada
    layers.Dropout(0.5),  # Dropout para regularización

    # Capa de salida para clasificación binaria
    layers.Dense(1, activation='sigmoid')  # Para clasificación binaria
])

In [18]:
sgd = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
# Compilar el modelo con el optimizador Adam y la función de pérdida adecuada
#model.compile(optimizer='adam',
#              loss='binary_crossentropy',  # Para clasificación binaria
#              metrics=['accuracy'])  # Métrica de precisión
# Compilar el modelo con el optimizador SGD
model.compile(optimizer=sgd,
              loss='binary_crossentropy',  # Para clasificación binaria
              metrics=['accuracy'])  # Métrica de precisión

In [19]:
# Obtener la salida del modelo para la imagen
resultados = model.predict(imagen_array)

# Si es una salida de predicción binaria, puedes interpretar el resultado
print("Predicción:", resultados[0])  # Muestra la salida de la predicción

1/1 [==============================] - 0s 121ms/step
Predicción: [0.49930352]


In [ ]:
import matplotlib.pyplot as plt
import os  # Para manipulación de directorios y archivos

# Asegúrate de que el directorio donde guardarás las imágenes exista
output_dir = "output_images"
os.makedirs(output_dir, exist_ok=True)  # Crea el directorio si no existe

# Número de filtros en la salida (la última dimensión)
num_filters = resultados.shape[-1]  # Cantidad de filtros

# Crear subgráficos, asegurándote de que sean iterables
fig, axs = plt.subplots(1, num_filters, figsize=(15, 15))  # Crea subgráficos para cada filtro

# Verificar la forma de `resultados` para asegurarse de que es correcta
if resultados.ndim == 4:  # Asegúrate de que tiene 4 dimensiones
    # Crear subgráficos, asegurándote de que sean iterables
    fig, axs = plt.subplots(1, resultados.shape[-1], figsize=(15, 15))  # Crea subgráficos para cada filtro

    # Asegúrate de que `axs` es iterable, incluso si solo hay un subgráfico
    if resultados.shape[-1] == 1:
        axs = [axs]  # Convertir a lista para poder indexar

    # Guardar cada filtro como una imagen
    for i in range(resultados.shape[-1]):  # Asegúrate de que estás en el rango correcto
        axs[i].imshow(resultados[0, :, :, i], cmap='viridis')  # `imshow` para visualizar
        axs[i].axis('off')  # Ocultar ejes para una mejor visualización

        # Guardar la imagen en el directorio especificado
        file_name = f"filtro_{i + 1}.png"
        fig.savefig(file_name, bbox_inches='tight')  # Guardar figura

    plt.close(fig)  # Cerrar la figura para liberar memoria
else:
    print("El array 'resultados' no tiene 4 dimensiones como se esperaba. Forma:", resultados.shape)


El array 'resultados' no tiene 4 dimensiones como se esperaba. Forma: (1, 1)
